In [1]:
from keras.datasets import mnist
from keras.layers import *
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras

import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
X_train.shape

(60000, 28, 28)

*There are 60000 mnist images used for Training and each image is of 28*28 Dimension as 2D Array*




In [4]:
y_train.shape

(60000,)

In [5]:
X_test.shape

(10000, 28, 28)

*There are 10000 mnist images used for Evaluation*


In [6]:
y_test.shape

(10000,)

In [7]:
# Feature Scaling
X_train = (X_train - 127.5) / 127.5

print(X_train.min())
print(X_train.max())

-1.0
1.0


In [8]:
EPOCHS = 100
BATCH_SIZE = 265
HALF_BATCH = 128

NO_OF_BATCHES = X_train.shape[0] // BATCH_SIZE
NO_OF_HALF_BATCHES = X_train.shape[0] // HALF_BATCH

NOISE_DIM = 100

adam  = Adam(learning_rate=2e-4, beta_1=0.5)

In [9]:
# Generator Model: Upsampling

generator = Sequential()
generator.add(Dense(units=7*7*128, input_shape=(NOISE_DIM, )))
generator.add(Reshape((7, 7, 128)))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

# Upsampling : (7, 7, 128) -> (14, 14, 64)

generator.add(Conv2DTranspose(filters=64, kernel_size=(5, 5), strides=(2, 2), padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

# Upsampling : (14, 14, 64) -> (28, 28, 1)

generator.add(Conv2DTranspose(filters=1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='tanh'))
generator.compile(loss=keras.losses.binary_crossentropy, optimizer=adam)
generator.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 6272)           │       633,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 14, 14, 64)     │       204,864 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 28, 28, 1)      │           577 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 839,681 (3.20 MB)

 Trainable params: 839,297 (3.20 MB)

 Non-trainable params: 384 (1.50 KB)

In [10]:
# Discriminator Model: Downsampling
# (28, 28, 1) -> (14, 14, 64)

descriminator = Sequential()
descriminator.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same', input_shape=(28, 28, 1)))
descriminator.add(LeakyReLU(0.2))

# (14, 14, 64) -> (7, 7, 128)

descriminator.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same'))
descriminator.add(LeakyReLU(0.2))

# (7, 7, 128) -> 6272
descriminator.add(Flatten())
descriminator.add(Dense(100))
descriminator.add(LeakyReLU(0.2))

descriminator.add(Dense(units=1, activation='sigmoid'))
descriminator.compile(loss=keras.losses.binary_crossentropy, optimizer=adam)
descriminator.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 14, 14, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │       627,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 701,897 (2.68 MB)

 Trainable params: 701,897 (2.68 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Combine the 2 models
descriminator.trainable = False
gan_input = Input(shape=(NOISE_DIM, ))
generated_img = generator(gan_input)
gan_output = descriminator(generated_img)

In [12]:
# Functional API
model = Model(gan_input, gan_output)
model.compile(loss=keras.losses.binary_crossentropy, optimizer=adam)
model.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 28, 28, 1)      │       839,681 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │       701,897 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,541,578 (5.88 MB)

 Trainable params: 839,297 (3.20 MB)

 Non-trainable params: 702,281 (2.68 MB)

In [13]:
# Reshape Training Input

X_train = X_train.reshape(-1, 28, 28, 1)

In [14]:
def display_images(samples= 256):
  noise = np.random.normal(0, 1, size=(samples, NOISE_DIM))
  generated_imgs = generator.predict(noise)

  plt.figure(figsize=(10, 10))

  for i in range(samples):
    plt.subplot(5, 5, i+1)
    plt.imshow(generated_imgs[i].reshape(28, 28), cmap='gray')
    plt.axis('off')

  plt.tight_layout()
  plt.show()

In [ ]:
# Training Loop
d_losses = []
g_losses = []

for epoch in range(EPOCHS):
  epoch_d_losses = 0.0
  epoch_g_losses = 0.0

  # Mini-Batch Gradient Descent
  for step in range(NO_OF_BATCHES):
    # Stage1: Train Descriminator
    descriminator.trainiable = True

    # Get the Real Data
    idx = np.random.randint(0, X_train.shape[0], HALF_BATCH)
    real_imgs = X_train[idx]

    # Get the Fake Data
    noise = np.random.normal(0, 1, size=(HALF_BATCH, NOISE_DIM))
    fake_imgs = generator.predict(noise)

    # Labels
    real_y = np.ones((HALF_BATCH, 1)) * 0.9
    fake_y = np.zeros((HALF_BATCH, 1))

    # Train Descriminator
    d_loss_real = descriminator.train_on_batch(real_imgs, real_y)
    d_loss_fake = descriminator.train_on_batch(fake_imgs, fake_y)

    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    epoch_d_losses += d_loss

    # ==============================================
    # Stage2: Train Generator
    generator.trainable = True

    noise = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_DIM))
    g_loss = model.train_on_batch(noise, np.ones((BATCH_SIZE, 1)) * 0.9)
    epoch_g_losses += g_loss


    print(f"Epoch {epoch+1}, Disc Loss: {epoch_d_losses / NO_OF_BATCHES}, Gen Loss: {epoch_g_losses / NO_OF_BATCHES}")

  d_losses.append(epoch_d_losses / NO_OF_BATCHES)
  g_losses.append(epoch_g_losses / NO_OF_BATCHES)

  if (epoch+1)%10 == 0:
    generator.save_weights(f"generator_{epoch+1}.h5")
    descriminator.save_weights(f"descriminator_{epoch+1}.h5")
    display_images(epoch+1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Epoch 1, Disc Loss: 0.002982672769576311, Gen Loss: 0.003014511661604047
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Epoch 1, Disc Loss: 0.005983678158372641, Gen Loss: 0.00601188512519002
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch 1, Disc Loss: 0.008988321758806705, Gen Loss: 0.008993694558739662
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch 1, Disc Loss: 0.011996072717010975, Gen Loss: 0.01195951271802187
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Epoch 1, Disc Loss: 0.015005458146333694, Gen Loss: 0.014911480247974396
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Epoch 1, Disc Loss: 0.01801678165793419, Gen Loss: 0.01784973405301571
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Epoch 1, Disc Loss: 0.021029533818364143, Gen Loss: 0.02077377401292324
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Epoch 1, Disc Loss: 0.024043191224336624, Gen Loss: 0.023683981969952583
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Epoch 1, Disc Loss: 0.02705812081694603, Gen Loss: 0.026579253375530243
4/4 ━━━